In [2]:
import os
import traceback
import numpy as np
import time
#from cuda import cuda, nvrtc

from hyperbolicTSNE.util import find_last_embedding
from hyperbolicTSNE.visualization import plot_poincare, animate
from hyperbolicTSNE import load_data, Datasets, SequentialOptimizer, initialization, HyperbolicTSNE

data_home = "datasets"
log_path = "temp/poincare/"  # path for saving embedding snapshots

only_animate = False
seed = 42
dataset = Datasets.PLANARIA  # the Datasets handler provides access to several data sets used throughout the repository
num_points = 10000  # we use a subset for demonstration purposes, full MNIST has N=70000
perp = 30  # we use a perplexity of 30 in this example

dataX, dataLabels, D, V, _ = load_data(
    dataset, 
    data_home=data_home, 
    random_state=seed, 
    to_return="X_labels_D_V",
    hd_params={"perplexity": perp}, 
    sample=num_points, 
    knn_method="hnswlib"  # we use an approximation of high-dimensional neighbors to speed up computations
)

print("Data loaded")

exaggeration_factor = 12  # Just like regular t-SNE, we use early exaggeration with a factor of 12
learning_rate = (dataX.shape[0] * 1) / (exaggeration_factor * 1000)  # We adjust the learning rate to the hyperbolic setting
ex_iterations = 250  # The embedder is to execute 250 iterations of early exaggeration, ...
main_iterations = 750  # ... followed by 750 iterations of non-exaggerated gradient descent.

opt_config = dict(
    learning_rate_ex=learning_rate,  # learning rate during exaggeration
    learning_rate_main=learning_rate,  # learning rate main optimization 
    exaggeration=exaggeration_factor, 
    exaggeration_its=ex_iterations, 
    gradientDescent_its=main_iterations, 
    vanilla=False,  # if vanilla is set to true, regular gradient descent without any modifications is performed; for  vanilla set to false, the optimization makes use of momentum and gains
    momentum_ex=0.5,  # Set momentum during early exaggeration to 0.5
    momentum=0.8,  # Set momentum during non-exaggerated gradient descent to 0.8
    exact=True,  # To use the quad tree for acceleration (like Barnes-Hut in the Euclidean setting) or to evaluate the gradient exactly
    area_split=False,  # To build or not build the polar quad tree based on equal area splitting or - alternatively - on equal length splitting
    n_iter_check=10,  # Needed for early stopping criterion
    size_tol=0.999  # Size of the embedding to be used as early stopping criterion
)

opt_params = SequentialOptimizer.sequence_poincare(**opt_config)

print("Sequence defined")

# Start: configure logging
logging_dict = {
    "log_path": log_path
}
opt_params["logging_dict"] = logging_dict

log_path = opt_params["logging_dict"]["log_path"]
# Delete old log path
if os.path.exists(log_path) and not only_animate:
    import shutil
    shutil.rmtree(log_path)
# End: logging

# Compute an initial embedding of the data via PCA
X_embedded = initialization(
    n_samples=dataX.shape[0],
    n_components=2,
    X=dataX,
    random_state=seed,
    method="pca"
)

# Initialize the embedder
htsne = HyperbolicTSNE(
    init=X_embedded, 
    n_components=2, 
    metric="precomputed", 
    verbose=True, 
    opt_method=SequentialOptimizer, 
    opt_params=opt_params
)

start_time = time.time()

try:
    hyperbolicEmbedding = htsne.fit_transform((D, V))
except ValueError:
    print("Error!")
    hyperbolicEmbedding = find_last_embedding(log_path)
    traceback.print_exc()

end_time = time.time()

execution_time = end_time - start_time
print("Execution time:", execution_time, "seconds")

# Create a rendering of the embedding and save it to a file
if not os.path.exists("results"):
    os.mkdir("results")
fig = plot_poincare(hyperbolicEmbedding, dataLabels)
fig.savefig(f"results/{dataset.name}.png")

Data loaded
Please note that `empty_sequence` uses the KL divergence with Barnes-Hut approximation (angle=0.5) by default.
Sequence defined
[HyperbolicTSNE] Received iterable as input. It should have len=2 and contain (D=None, V=None)
[hd_mat] Warning: There is nothing to do with given parameters. Returning given D and V
Running Gradient Descent, Verbosity: True
[gradient_descent] Warning!: because of logging, the cf will be computed at every iteration


Gradient Descent:   0%|          | 0/250 [00:00<?, ?it/s]

Mem alloc:  0.0004980564117431641 seconds
Mem copy:  0.0004990100860595703 seconds
Run:  0.0 seconds


Gradient Descent error: 93.98324 grad_norm: 4.27717e-01:   0%|          | 1/250 [00:00<01:49,  2.28it/s]

Copy back:  0.28504419326782227 seconds
Total:  0.28653979301452637 seconds
sumQ:  99999984.56039003
neg_f took  0.28700000000000003 s
pos_f took  0.048 s
applying forces took  0.01 s
Mem alloc:  0.0004982948303222656 seconds
Mem copy:  0.0004987716674804688 seconds
Run:  0.0 seconds


...

Gradient Descent error: 93.96389 grad_norm: 8.42147e-01:  28%|██▊       | 71/250 [00:24<01:03,  2.82it/s]

Copy back:  0.15375423431396484 seconds
Total:  0.15525317192077637 seconds
sumQ:  99792836.40880847
neg_f took  0.156 s
pos_f took  0.059000000000000004 s
applying forces took  0.007 s
Mem alloc:  0.0 seconds
Mem copy:  0.0 seconds
Run:  0.0 seconds


Gradient Descent error: 93.96463 grad_norm: 8.38867e-01:  29%|██▉       | 72/250 [00:24<01:01,  2.87it/s]

Copy back:  0.16273736953735352 seconds
Total:  0.16373801231384277 seconds
sumQ:  99787639.10450326
neg_f took  0.164 s
pos_f took  0.058 s
applying forces took  0.012 s
Mem alloc:  0.0004985332489013672 seconds
Mem copy:  0.0 seconds
Run:  0.0 seconds


Gradient Descent error: 93.96188 grad_norm: 8.41721e-01:  29%|██▉       | 72/250 [00:24<01:01,  2.87it/s]